In [1]:
import glob

import numpy as np

from PIL import Image

import coremltools

/Users/vovacher/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/vovacher/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/vovacher/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/vovacher/anaconda3/lib/python3.7/site-packages/tensorflow/python/f

# Load CoreML model

In [2]:
model_fp32 = coremltools.models.MLModel(
    f"../vision/classification/resnet_v1/models/resnet18_v1_torchvision.mlmodel"
)

# Quantize CoreML model

The following quantizations are [supported](https://apple.github.io/coremltools/generated/coremltools.models.neural_network.quantization_utils.html#coremltools.models.neural_network.quantization_utils.quantize_weights):
* **FP16**
* **1-8 bit**

In [3]:
model_fp16 = coremltools.utils.convert_neural_network_weights_to_fp16(model_fp32)

Quantizing layer input_1_scaler_custom
Quantizing layer Conv_0
Quantizing layer BatchNormalization_1
Quantizing layer Conv_4
Quantizing layer BatchNormalization_5
Quantizing layer Conv_7
Quantizing layer BatchNormalization_8
Quantizing layer Conv_11
Quantizing layer BatchNormalization_12
Quantizing layer Conv_14
Quantizing layer BatchNormalization_15
Quantizing layer Conv_18
Quantizing layer BatchNormalization_19
Quantizing layer Conv_21
Quantizing layer BatchNormalization_22
Quantizing layer Conv_23
Quantizing layer BatchNormalization_24
Quantizing layer Conv_27
Quantizing layer BatchNormalization_28
Quantizing layer Conv_30
Quantizing layer BatchNormalization_31
Quantizing layer Conv_34
Quantizing layer BatchNormalization_35
Quantizing layer Conv_37
Quantizing layer BatchNormalization_38
Quantizing layer Conv_39
Quantizing layer BatchNormalization_40
Quantizing layer Conv_43
Quantizing layer BatchNormalization_44
Quantizing layer Conv_46
Quantizing layer BatchNormalization_47
Quantiz

In [4]:
model_8bit = coremltools.models.neural_network.quantization_utils.quantize_weights(model_fp32, 
                                                                                   nbits=8, 
                                                                                   quantization_mode="linear")

Quantizing using linear quantization
Optimizing Neural Network before Quantization:
Fused Conv_0->BatchNormalization_1
Fused Conv_4->BatchNormalization_5
Fused Conv_7->BatchNormalization_8
Fused Conv_11->BatchNormalization_12
Fused Conv_14->BatchNormalization_15
Fused Conv_18->BatchNormalization_19
Fused Conv_21->BatchNormalization_22
Fused Conv_23->BatchNormalization_24
Fused Conv_27->BatchNormalization_28
Fused Conv_30->BatchNormalization_31
Fused Conv_34->BatchNormalization_35
Fused Conv_37->BatchNormalization_38
Fused Conv_39->BatchNormalization_40
Fused Conv_43->BatchNormalization_44
Fused Conv_46->BatchNormalization_47
Fused Conv_50->BatchNormalization_51
Fused Conv_53->BatchNormalization_54
Fused Conv_55->BatchNormalization_56
Fused Conv_59->BatchNormalization_60
Fused Conv_62->BatchNormalization_63
Finished optimizing network. Quantizing neural network..
Quantizing layer input_1_scaler_custom
Quantizing layer Conv_0
Quantizing layer Conv_4
Quantizing layer Conv_7
Quantizing lay

# Evaluate

In [5]:
test_filelist = sorted(glob.glob("../data/images/test/*.jpg"))
len(test_filelist)

10

In [6]:
preds_fp32_all = []
preds_fp16_all = []
preds_8bit_all = []
for file in test_filelist:
    _image = Image.open(file)
    _image = _image.resize((224, 224))

    _preds_fp32 = model_fp32.predict({"input_1": _image}, useCPUOnly=True)["output_1"]
    _preds_fp32_array = []
    for cl in sorted(_preds_fp32.keys()):
        _preds_fp32_array.append(_preds_fp32[cl])
    preds_fp32_all.append(np.array([_preds_fp32_array], dtype="float32"))

    _preds_fp16 = model_fp16.predict({"input_1": _image}, useCPUOnly=True)["output_1"]
    _preds_fp16_array = []
    for cl in sorted(_preds_fp32.keys()):
        _preds_fp16_array.append(_preds_fp16[cl])
    preds_fp16_all.append(np.array([_preds_fp16_array], dtype="float32"))

    _preds_8bit = model_8bit.predict({"input_1": _image}, useCPUOnly=True)["output_1"]
    _preds_8bit_array = []
    for cl in sorted(_preds_fp32.keys()):
        _preds_8bit_array.append(_preds_8bit[cl])
    preds_8bit_all.append(np.array([_preds_8bit_array], dtype="float32"))

preds_fp32_all = np.concatenate(preds_fp32_all)
preds_fp16_all = np.concatenate(preds_fp16_all)
preds_8bit_all = np.concatenate(preds_8bit_all)

FP32 (full)

In [7]:
np.median(abs(preds_fp32_all - preds_fp32_all) / preds_fp32_all)

0.0

FP16

In [8]:
np.median(abs(preds_fp32_all - preds_fp16_all) / preds_fp32_all)

0.006121083

8-bit quantization

In [9]:
np.median(abs(preds_fp32_all - preds_8bit_all) / preds_fp32_all)

0.12689283